In [ ]:
#Load the data set:
#necessary imports
import torch
import pandas as pd 
from torch.utils.data import Dataset, DataLoader

dataset = pd.read_tsv('eestec_hackathon_2025_train.tsv' ,sep = '\t',names=['ID', 'Label', 'Statement', 'Subjects', 'Speaker Name', 'Speaker Title', 'State', 'Party Affiliation', 'Credit History: barely-true', 'Credit History: false', 'Credit History: half-true', 'Credit History: mostly-true', 'Credit History: pants-fire', 'Context/Location'])

In [ ]:
import csv
import pandas as pd 
import torch
import torchtext 
from torchtext.data import Field, TabularDataset

Statement = Field(sequential =  True, lower = True, tokenize = "spacy", stop_words = "english")

 
